In [ ]:
import torch
from resnet import resnet56
from embedl.plumbing.torch.metrics.target import Target
from embedl.torch.pruning.methods import UniformPruning
from embedl.torch.viewer import view_model
from embedl.torch.metrics.performances import Flops  
from embedl.torch.metrics.measure_performance import measure_flops

In [ ]:
model = resnet56()
input_shape = [1,3,32,32]       
base_flops = measure_flops(
    model=model,
    input_shape=input_shape
)
pruning_method = UniformPruning(
    target=Target(Flops(), fraction=0.6),
    step_size=1
)
pruning_steps = pruning_method.prune(model, input_shape)
pruned_flops = measure_flops(
    model=model,
    input_shape=input_shape
)
print(pruned_flops/base_flops) # 0.5976937173922103
# torch.save(model, 'resnet56_pruned/resnet56_uniform_60.th')

In [ ]:
view_model(model,input_shape,compare_with=resnet56())